In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue

In [2]:
###QUESTIONS TO TA 
Need to implement bad init & L2/L2 penalties on other files ?

SyntaxError: invalid syntax (<ipython-input-2-9ac5deb0902f>, line 2)

Generate pairs

In [3]:
N = 1000 #nb of pairs

#generate pairs
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [4]:
#normalize the input
train_input/=255
test_input/=255

In [5]:
new_train_target = torch.empty(N,2)
new_test_target = torch.empty(N,2)
for i in range(N):
    if train_target[i] == 1 :
        new_train_target[i,0] = 0
        new_train_target[i,1] = 1
        
    else:
        new_train_target[i,0] = 1
        new_train_target[i,1] = 0
        
    if test_target[i] == 1:
        new_test_target[i,0] = 0
        new_test_target[i,1] = 1
        
    else:
        new_test_target[i,0] = 1
        new_test_target[i,1] = 0

In [6]:
def create_shallow_model():
    return nn.Sequential(
        nn.Linear(392, 400),
        nn.ReLU(),
        nn.Linear(400, 500),
        nn.ReLU(),
        nn.Linear(500, 600),
        nn.ReLU(),
        nn.Linear(600, 2),
    )

In [7]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(392, 784, kernel_size=4)
        self.conv2 = nn.Conv1d(784, 1568, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [8]:
def train_model(model, train_input, train_target,lr):
    epoch = 25
    eta = 0.2
    mini_batches = 100
    optimizer = optim.Adam(model.parameters(), lr)
    
    criterion = nn.BCEWithLogitsLoss()
    
    for e in range(epoch):
        sum_loss = 0
        
        for b in range(0, train_input.size(0), mini_batches):
            
            output = model(train_input.narrow(0, b, mini_batches).reshape(mini_batches, 1, -1))
            #print('shapes',output.squeeze(1).shape, train_target.narrow(0, b, mini_batches).shape)
            loss = criterion(output.squeeze(1), train_target.narrow(0, b, mini_batches))
            loss.requires_grad_()
            #print("output", output.squeeze(1), "train", train_target.narrow(0, b, mini_batches))
            model.zero_grad()
            #print("output", output, "shape", output.shape)
            loss.backward()
            optimizer.step()
            sum_loss = sum_loss + loss.item()
            
            """#print("LOSS",loss.item())
            with torch.no_grad():
                for p in model.parameters():
                    p -= eta*p.grad
                    #print("grads", p.grad)"""
        print(e, sum_loss)


In [9]:
def compute_nb_errors(model, test_input, target):
    nb_errors = 0
    mini_batch_size = 100
    
    for b in range(0, test_input.size(0), mini_batch_size):
        output = model(test_input.narrow(0, b, mini_batch_size).reshape(mini_batch_size, 1, -1))
        _, predicted_class = output.max(2)
        #print(output)
        #print(predicted_class, output, target)
        #print("pred classes",predicted_class.shape, "output", output.shape, "target", target.shape)
        for k in range(mini_batch_size):
            
            if target[b + k, predicted_class[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [10]:
##########################bad initialization (gaussian)
lr = 0.01

for std in [ -1, 1e-3, 1e-2, 1e-1, 1e-0, 1e1 ]:
    model = create_shallow_model()
    if std > 0:
        with torch.no_grad():
            for p in model.parameters(): p.normal_(0, std)
    train_model(model, train_input, new_train_target, lr)
    nb_train_errors = compute_nb_errors(model, train_input, new_train_target)
    nb_test_errors = compute_nb_errors(model, test_input, new_test_target)
    print("STD :", std)
    print('train error {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                       nb_train_errors, train_input.size(0)))

    print('test error {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 8.402923345565796
1 5.945102274417877
2 4.912848860025406
3 3.8082814812660217
4 3.276098310947418
5 2.627827078104019
6 2.5889915376901627
7 2.188942566514015
8 2.0065199583768845
9 1.7439365535974503
10 1.7917822822928429
11 1.6822655722498894
12 1.8172352313995361
13 2.0755162686109543
14 1.6414236351847649
15 2.259162649512291
16 2.626695439219475
17 2.4141112118959427
18 2.0306322127580643
19 1.6831796914339066
20 1.6199790760874748
21 1.7704265266656876
22 1.1476543918251991
23 0.842155896127224
24 0.5380828846246004
STD : -1
train error 1.80%% 18/1000
test error 22.50%% 225/1000
0 7.201122462749481
1 6.239093840122223
2 5.026223450899124
3 4.59284970164299
4 4.002883940935135
5 3.7058696150779724
6 3.3506076335906982
7 3.188093513250351
8 3.497215837240219
9 2.542859375476837
10 2.392606094479561
11 2.1573038697242737
12 2.062415897846222
13 2.746184542775154
14 2.046277403831482
15 1.7751050144433975
16 1.9736983999609947
17 2.1922746300697327
18 3.6917563676834106
19 3.68690

In [11]:
# good result : lr = 0.01 : no overfitting, good test error ~20%

################good initialization (automatic from pytorch)

for lr in [0.001,0.005, 0.01, 0.05, 0.1]:
    model = create_shallow_model()
    train_model(model, train_input, new_train_target, lr)
    nb_train_errors = compute_nb_errors(model, train_input, new_train_target)
    nb_test_errors = compute_nb_errors(model, test_input, new_test_target)
    print("Learning rate :", lr)
    print('train error {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                       nb_train_errors, train_input.size(0)))

    print('test error {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 6.625135898590088
1 4.947438418865204
2 4.169629544019699
3 3.2450728565454483
4 2.4105163365602493
5 1.6404950469732285
6 1.7230835482478142
7 0.69340805336833
8 0.7366314390674233
9 0.45100995246320963
10 0.8801031894981861
11 2.79973151255399
12 3.566630057990551
13 2.035114750266075
14 1.0744341686367989
15 0.5557735962793231
16 0.1700974451377988
17 0.059785716934129596
18 0.017462995601817966
19 0.008908532268833369
20 0.005080530099803582
21 0.0033747516135917976
22 0.0023519350434071384
23 0.001761031620844733
24 0.0013520521897589788
Learning rate : 0.001
train error 0.00%% 0/1000
test error 18.50%% 185/1000
0 5.998960167169571
1 5.243988454341888
2 4.012353897094727
3 2.954407900571823
4 2.285946637392044
5 2.193681515753269
6 1.6064449548721313
7 0.9908334277570248
8 0.8561000153422356
9 1.0393062457442284
10 0.4872272238135338
11 0.3219771031290293
12 0.894184130243957
13 0.5746839493513107
14 1.2253377921879292
15 0.664965683594346
16 1.2762038130313158
17 1.445819754153

In [12]:
def train_model_penalties(model, train_input, train_target,lr, L1, value): #L1 = 1 if L1 // =0 if L2
    epoch = 25
    eta = 0.2
    mini_batches = 100
    optimizer = optim.Adam(model.parameters(), lr)
    
    criterion = nn.BCEWithLogitsLoss()
    
    for e in range(epoch):
        sum_loss = 0
        
        for b in range(0, train_input.size(0), mini_batches):
            
            output = model(train_input.narrow(0, b, mini_batches).reshape(mini_batches, 1, -1))
            loss = criterion(output.squeeze(1), train_target.narrow(0, b, mini_batches))
            
            if L1 == 1:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                with torch.no_grad():
                    for p in model.parameters():
                        p.sub_(p.sign() * p.abs().clamp(max = value))

            else:
                for p in model.parameters():
                    sum_loss += value * p.pow(2).sum()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        if not L1:
            print(e, sum_loss)

In [13]:
########L2 penalty
lr = 0.01
for lambda_l2 in [0.001, 0.002, 0.004, 0.005, 0.010, 0.020]:
    model = create_shallow_model()
    train_model_penalties(model, train_input, new_train_target, lr, 0, lambda_l2)
    nb_train_errors = compute_nb_errors(model, train_input, new_train_target)
    nb_test_errors = compute_nb_errors(model, test_input, new_test_target)
    print("L2 :", lambda_l2)
    print('train error {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                       nb_train_errors, train_input.size(0)))

    print('test error {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 tensor(8.1133, grad_fn=<AddBackward0>)
1 tensor(15.2108, grad_fn=<AddBackward0>)
2 tensor(19.3217, grad_fn=<AddBackward0>)
3 tensor(21.3958, grad_fn=<AddBackward0>)
4 tensor(22.6093, grad_fn=<AddBackward0>)
5 tensor(23.3564, grad_fn=<AddBackward0>)
6 tensor(24.0225, grad_fn=<AddBackward0>)
7 tensor(24.6368, grad_fn=<AddBackward0>)
8 tensor(25.1696, grad_fn=<AddBackward0>)
9 tensor(25.4054, grad_fn=<AddBackward0>)
10 tensor(26.0577, grad_fn=<AddBackward0>)
11 tensor(26.9290, grad_fn=<AddBackward0>)
12 tensor(27.5305, grad_fn=<AddBackward0>)
13 tensor(27.8460, grad_fn=<AddBackward0>)
14 tensor(28.0629, grad_fn=<AddBackward0>)
15 tensor(28.3894, grad_fn=<AddBackward0>)
16 tensor(28.7183, grad_fn=<AddBackward0>)
17 tensor(28.9581, grad_fn=<AddBackward0>)
18 tensor(29.4380, grad_fn=<AddBackward0>)
19 tensor(30.0903, grad_fn=<AddBackward0>)
20 tensor(30.6240, grad_fn=<AddBackward0>)
21 tensor(31.0830, grad_fn=<AddBackward0>)
22 tensor(31.7106, grad_fn=<AddBackward0>)
23 tensor(32.4900, gra

In [14]:
########L1 penalty
lr = 0.01
for lambda_l1 in [0.001, 0.002, 0.004, 0.005, 0.010, 0.020]:
    model = create_shallow_model()
    train_model_penalties(model, train_input, new_train_target, lr, 1, lambda_l1)
    nb_train_errors = compute_nb_errors(model, train_input, new_train_target)
    nb_test_errors = compute_nb_errors(model, test_input, new_test_target)
    print("L1 :", lambda_l1)
    print('train error {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                       nb_train_errors, train_input.size(0)))

    print('test error {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

L1 : 0.001
train error 5.50%% 55/1000
test error 24.10%% 241/1000
L1 : 0.002
train error 7.30%% 73/1000
test error 20.80%% 208/1000
L1 : 0.004
train error 8.80%% 88/1000
test error 21.70%% 217/1000
L1 : 0.005
train error 10.00%% 100/1000
test error 23.20%% 232/1000
L1 : 0.01
train error 44.90%% 449/1000
test error 47.40%% 474/1000
L1 : 0.02
train error 44.90%% 449/1000
test error 47.40%% 474/1000
